In [30]:
import pandas as pd
import datetime

df = pd.read_excel('Отчет_доставки_по_Курьерам_14_06_2022_16_06_22.xlsx')
df = df[(df['Тип заказа']!='На время самовывоз') & (df['Тип заказа']!='Доставка самовывоз') & (df['Тип заказа']!='Самовывоз')]

for i in df['Курьер'].unique():
    print('Курьер: ', i)
    df_courier = df[df['Курьер'] == i]
    orders = df_courier.shape[0]
    print('Отвезенных заказов всего: ', orders)
    dovozi = df_courier[(df_courier['Сумма со скидкой, р.'] < 200) & (df_courier['Сумма со скидкой, р.'] > 0)]
    dovozi_amount = 0    

    for z in range(dovozi.shape[0]):
        if df_courier[(df_courier['Адрес'] == dovozi.iloc[z]['Адрес']) \
        & (df_courier['Фактическое время доставки'] <= (dovozi.iloc[z]['Фактическое время доставки']) + datetime.timedelta(minutes=3)) \
        & (df_courier['Фактическое время доставки'] >= (dovozi.iloc[z]['Фактическое время доставки']) - datetime.timedelta(minutes=3))].shape[0] > 1:
            df_courier = df_courier[df_courier['Номер чека']!= dovozi.iloc[z]['Номер чека']]
            dovozi_amount = dovozi_amount + 1
    
    print('Количество довозов без оплаты: ', dovozi_amount)
    orders_next = df_courier.shape[0]
    print('Отвезенных заказов без довозов: ', orders_next)
    
    arr = []
    for j in range(orders_next):
        reis = df_courier[(df_courier['Время отправки доставки'] < (df_courier.iloc[j]['Время отправки доставки'] + datetime.timedelta(minutes=20))) \
        & (df_courier['Время отправки доставки'] > (df_courier.iloc[j]['Время отправки доставки'] - datetime.timedelta(minutes=20)))].shape[0]
        arr = arr + [reis]  
    
    dict_ = {}
    for q in set(arr):
        dict_[q] = arr.count(q)/q 
    dict_ = dict(sorted(dict_.items()))
    print(dict_)

    print('Среднее количество заказов в рейсе: ',  orders_next / sum(dict_.values()))


Курьер:  Гришин Иван
Отвезенных заказов всего:  24
Количество довозов без оплаты:  1
Отвезенных заказов без довозов:  23
{1: 16.0, 2: 2.0, 3: 1.0}
Среднее количество заказов в рейсе:  1.2105263157894737


In [1]:
#prepearing date for analisis
import pandas as pd

adress_control['Время открытия'] = adress_control['Время открытия'].fillna(method='ffill')
adress_control = adress_control[(adress_control['Тип заказа']!='На время самовывоз') & (adress_control['Тип заказа']!='Доставка самовывоз') & (adress_control['Тип заказа']!='Самовывоз')]
display(adress_control)
adress_control.to_excel('доставки по курьерам/Отчет_доставки_по_Курьерам_14_06_2022_16_06_22.xlsx')

,Торговое предприятие,Время открытия,Время печати накладной,Время отправки доставки,Планируемое время доставки,Фактическое время доставки,Время закрытия,Курьер,Время в пути(мин),Город,Адрес,Тип заказа,Возврат чека,Номер чека,Заказов,Ср.время в пути(мин),Сумм.время в пути(мин),Ср.опоздание доставки(мин),"Сумма со скидкой, р."
0,Алроса,2022-06-14 10:23:11,2022-06-14 13:19:15.342,2022-06-14 13:21:58.147,2022-06-14 14:00:40.039,2022-06-14 13:48:47,2022-06-14 13:19:16.153,Евгений Егоров,26,Орел,ул. 2-я Посадская д. 4 кв./оф. 41 под. 2 эт. 4,На время курьером,Нет,123823,1,26,26,0,1299
1,Алроса,2022-06-14 10:40:50,2022-06-14 10:55:15.849,2022-06-14 11:02:29.700,2022-06-14 11:50:22.000,2022-06-14 11:13:38,2022-06-14 10:55:17.989,Конценебин Михаил,11,Орел,ул. Емлютина д. 12 кв./оф. 230 под. 6 эт. 5 дф...,Доставка курьером,Нет,123825,1,11,11,0,822
2,Алроса,2022-06-14 10:41:50,2022-06-14 12:10:11.339,2022-06-14 12:21:43.469,2022-06-14 13:00:01.000,2022-06-14 12:49:34,2022-06-14 12:10:12.338,Евгений Егоров,27,Орел,ул. Машкарина д. 10 кв./оф. 68 под. 2 эт. 8,На время курьером,Нет,123826,1,27,27,0,1079
3,Алроса,2022-06-14 10:44:52,2022-06-14 11:02:17.125,2022-06-14 11:02:28.025,2022-06-14 11:52:58.000,2022-06-14 11:25:56,2022-06-14 11:02:17.831,Конценебин Михаил,23,Орел,ул. Комсомольская д. 44 кв./оф. 27 под. 2 эт. ...,Доставка курьером,Нет,123827,1,23,23,0,950
4,Алроса,2022-06-14 11:33:12,2022-06-14 11:44:06.923,2022-06-14 11:44:29.946,2022-06-14 12:40:33.000,2022-06-14 12:06:28,2022-06-14 11:44:07.791,Евгений Егоров,21,Орел,ул. Орелстроевская д. 11А кв./оф. 145 под. 4 э...,Доставка курьером,Нет,123828,1,21,21,0,925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,Раздольная,2022-06-16 20:28:07,2022-06-16 20:45:48.373,2022-06-16 20:49:18.464,2022-06-16 21:28:03.000,2022-06-16 21:20:20,2022-06-16 20:45:49.907,Жилябин Даниил,31,Орел,ул. Раздольная д. 27А кв./оф. 316 под. 3 эт. 8,Доставка курьером,Нет,58766,1,31,31,0,1653
603,Раздольная,2022-06-16 20:44:44,2022-06-16 20:59:24.820,2022-06-16 21:06:02.046,2022-06-16 21:54:22.000,2022-06-16 21:24:36,2022-06-16 20:59:27.877,Пашков Виталий,18,Орел,ул. Московская д. 66 кв./оф. 32 под. 2 эт. 2,Доставка курьером,Нет,58767,1,18,18,0,787
604,Раздольная,2022-06-16 20:47:16,2022-06-16 21:07:02.150,2022-06-16 21:32:06.870,2022-06-16 21:56:29.000,2022-06-16 21:43:29,2022-06-16 21:07:07.790,Жилябин Даниил,11,Жилина,ул. Строительная д. 15 кв./оф. 21 под. 3 эт. 1,Доставка курьером,Нет,58768,1,11,11,0,1073
605,Раздольная,2022-06-16 20:58:21,2022-06-16 21:25:32.373,2022-06-16 21:33:15.809,2022-06-16 22:08:03.000,2022-06-16 21:41:21,2022-06-16 21:25:34.358,Черников Дмитрий,8,д. Коневка,ул. Лесная аллея д. 1 корп./стр. 2 кв./оф. 160...,Доставка курьером,Нет,58769,1,8,8,0,822


In [ ]:
pip install --upgrade requests

In [ ]:
pip install -U ruamel.yaml

In [2]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import requests
import time
import pandas as pd
from openpyxl import load_workbook

In [2]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import requests
import time
import pandas as pd
from openpyxl import load_workbook

pd.set_option('display.max_columns', None, 'display.max_rows', None)
pd.options.mode.chained_assignment = None

adress_control = pd.read_excel('доставки по курьерам/Отчет_доставки_по_Курьерам_14_06_2022_16_06_22.xlsx')
adress_control.columns = adress_control.columns.str.lower()
adress_control.columns = adress_control.columns.str.strip()

#adress_control = adress_control.drop(adress_control[adress_control['тип заказа'] == 'На время самовывоз'].index)
#adress_control = adress_control.drop(adress_control[adress_control['тип заказа'] == 'Доставка самовывоз'].index)

counter = 0

def take_short_adress(x):
    per = ''
    if (len(x.split('д. ')[1].split()) > 1): 
        if (len(x.split('д. ')[1].split()[1]) == 1):
            per =  x.split('д. ')[1].split()[0] + x.split('д. ')[1].split()[1]
        else:
            per =  x.split('д. ')[1].split()[0]
    else:
        per = x.split('д. ')[1].split()[0]
    
    if ('корп./стр.' in list(x.split())):
        per = per + ' ' + x.split('корп./стр.')[1].split()[0]
     
    adress = x.split('д. ')[0] + 'д. ' + per
    return adress

def save_table_coord(df, sheet):
    writer = pd.ExcelWriter('courier_with_lat_and_lang.xlsx', engine='openpyxl')
    writer.book = load_workbook('courier_with_lat_and_lang.xlsx')   
    writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
    reader = pd.read_excel(r'courier_with_lat_and_lang.xlsx', sheet_name=sheet)
    df.to_excel(writer, sheet_name=sheet, index=False, header=False, startrow=len(reader)+1)
    writer.close() 

    
def take_coord1(x):
    DF_list_for_addition = list(x) + ['111'] 
    DF_for_addition = pd.DataFrame([DF_list_for_addition])
    save_table_coord(DF_for_addition, 'Sheet1')
    

def take_coord(x):
    global counter
    counter += 1 
    if counter < 995:
        key_api = '967d5b0c-1f61-4eea-83f7-888be714c3a2'
    elif (counter >= 995) and (counter < 1996):
        key_api = 'edc75c02-4b15-4be0-98fa-76d1ad08dce4'
   # else:
    #    key_api = 'edc75c02-4b15-4be0-98fa-76d1ad08dce4'
    print(counter)
    time.sleep(1)
    response = requests.get("https://geocode-maps.yandex.ru/1.x/?apikey=" + key_api + "&geocode=" + x['адрес B SF'])
    soup = BeautifulSoup(response.text,'lxml')
    if (soup.find('pos')):
        DF_list_for_addition = list(x) + [soup.find('pos').text]
        DF_for_addition = pd.DataFrame([DF_list_for_addition])
        save_table_coord(DF_for_addition, 'Sheet1')
        return soup.find('pos').text
    else:
        DF_list_for_addition = list(x) + ['адреса ошибка']
        DF_for_addition = pd.DataFrame([DF_list_for_addition])
        save_table_coord(DF_for_addition, 'Sheet1')
        return 'адреса ошибка'

def add_gorod(x):
    if x['город'] == 'Орел':
        return 'г. Орел ' + x['адрес']
    else:
        return 'Орловская область Орловский район ' + x['город'] + ' ' +  x['адрес']

adress_control['время отправки доставки'].iloc[0].month
adress_control['день + месяц'] = adress_control['время отправки доставки'].apply(lambda x: str(x.day) + '.' + str(x.month) + '.' + str(x.year))
adress_control['день + месяц'].unique()
adress_control['адрес А'] = 0
adress_control['адрес В'] = adress_control.apply(add_gorod, axis=1) 

adress_control['адрес B SF'] = adress_control['адрес В'].apply(take_short_adress)
adress_control['coords B'] = adress_control.apply(take_coord, axis=1)

#adress_control['coords B'] = adress_control['coords B'].apply(lambda x: x.split()[1] + ' ' + x.split()[0])
#adress_control.to_excel('courier_with_lat_and_lang.xlsx')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import requests
import time
import pandas as pd
from openpyxl import load_workbook
import numpy as np 


# запись ошибочный адресов для контрля
adress_control = pd.read_excel('courier_with_lat_and_lang.xlsx')
writer = pd.ExcelWriter('ошибки адреса.xlsx', engine='openpyxl')
writer.book = load_workbook('ошибки адреса.xlsx')   
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(r'courier_with_lat_and_lang.xlsx')
df1 = adress_control[adress_control['coords B'] == 'адреса ошибка']
df1.to_excel(writer, index=False, header=False, startrow=len(reader)+1)
writer.close() 

In [2]:
# проверка ошибочных адресов
adress_control = pd.read_excel('courier_with_lat_and_lang.xlsx')

if adress_control[adress_control['coords B'] == 'адреса ошибка'].size == 0:
    print('Without errors')
else:
    print('Warning! Before exceed next module, please fix adress errors')
    display(adress_control[adress_control['coords B']== 'адреса ошибка'])

Warning! Before exceed next module, please fix adress errors


,index,Торговое предприятие,Время открытия,Время печати накладной,Время отправки доставки,Планируемое время доставки,Фактическое время доставки,Время закрытия,Курьер,Время в пути(мин),...,Заказов,Ср.время в пути(мин),Сумм.время в пути(мин),Ср.опоздание доставки(мин),"Сумма со скидкой, р.",день + месяц,адрес A,адрес B,адрес B SF,coords B
445,542,Раздольная,2022-06-15 20:39:44,2022-06-15 21:09:09.903,2022-06-15 21:35:29.471,2022-06-15 21:48:01.778,2022-06-15 21:48:17,2022-06-15 21:09:11.830,Черников Дмитрий,12,...,1,12,12,0,1168,15.6.2022,0,г. Орел ул. Северный парк д. 8А,г. Орел ул. Северный парк д. 8А,адреса ошибка


In [3]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import requests
import time
import pandas as pd
from openpyxl import load_workbook
import numpy as np 

t_points = ['52.951462 36.025452', '52.978591 36.075605', '53.014416 36.133214']
adress_control = pd.read_excel('courier_with_lat_and_lang.xlsx')
adress_control.columns = adress_control.columns.str.lower()
adress_control.columns = adress_control.columns.str.strip()
adress_control['адрес a lba'] = 0
adress_control['coords b'] = adress_control['coords b'].apply(lambda x: x.split()[1] + ' ' + x.split()[0])
adress_control_concat_table = pd.DataFrame()
finish_courier_trip = pd.DataFrame(columns=['trip', 'courier', 'day'])
otchet_about_couriers = pd.DataFrame(columns=['Координаты', 'Маршрут','kourier','day'])
display(adress_control.head())


def create_mass(slice_list, slice_list_A_B, kourier, day):
    slice_list_temp = slice_list.copy()
    #display(slice_list_A_B[0])
    slice_list_temp_A_B = slice_list_A_B
    slice_list_full = []
    slice_list_full_A_B = []
    tt=0
    l = len(slice_list_temp)

    for i in range(l):
        if i==0: 
            tt = slice_list_temp[0][0]
            slice_list_full = slice_list_full + [slice_list_temp[0]]
            slice_list_full_A_B = slice_list_full_A_B + [slice_list_temp_A_B[0]]
            continue

        if slice_list_temp[i][0] in t_points:
            tt = slice_list_temp[i][0]
            slice_list_full = slice_list_full + [[slice_list_temp[i-1][1],slice_list_temp[i][0]]] 
            slice_list_full_A_B = slice_list_full_A_B + [[slice_list_temp_A_B[i-1][1], slice_list_temp_A_B[i][0]]] 
            
        slice_list_full = slice_list_full + [slice_list_temp[i]] 
        slice_list_full_A_B = slice_list_full_A_B + [slice_list_temp_A_B[i]] 
    #print(slice_list_temp)
    slice_list_full = slice_list_full + [[slice_list_temp[-1][1], tt]]
    
    if tt == '52.951462 36.025452':
        tt = 'Алроса'
    elif tt == '52.978591 36.075605':
        tt = 'Горького'
    elif tt == '53.014416 36.133214':
        tt = 'Раздольная'
    
    slice_list_full_A_B = slice_list_full_A_B + [[slice_list_temp_A_B[-1][1], tt]]
    # display(slice_list_full_A_B)
    distance_array = create_route(slice_list_full, kourier, day)
    
    #display(slice_list_full)
    #display(distance_array)
    df_otchet = pd.DataFrame(list(zip(slice_list_full, slice_list_full_A_B, distance_array)), columns =['Координаты', 'Адреса' , 'Маршрут'])
    df_otchet['kourier'] = kourier 
    df_otchet['day'] = day 
    return df_otchet

def create_route(slice_list, kourier, day):
    global finish_courier_trip 
    browser = Chrome('/Users/user/kvikfud/chromedriver_win32/chromedriver')
    roat_distance = 0
    list_roat_distance = []
    distance_array = []
    #display(slice_list)
    for A,B in slice_list:
        print('internal array')
        e = ''
        delay = 5
        flag = 0
        while e == '':
            try:
                url = "https://www.mapdevelopers.com/distance_from_to.php?&from=" + A + '&to=' + B
                browser.get(url)
                sleep(delay)
                if ((float(browser.find_element_by_id("driving_status").text.split()[5])==0) and (flag < 2)):
                    print('Нулевая дистанция')
                    flag = flag + 1
                else:
                    distance_array.append(browser.find_element_by_id("driving_status").text.split()[5])
                    e = ' '         
                
            except NoSuchElementException:
                sleep(10)
                print('Connection refused. Retrying...')
                
            except TimeoutException:
                sleep(10)
                print('TimeoutException Connection refused. Retrying...')
               
                        
        roat_distance = roat_distance + float(browser.find_element_by_id("driving_status").text.split()[5])
    
    list_roat_distance = [roat_distance] + [kourier] + [day]
    finish_courier_trip = finish_courier_trip.append(pd.DataFrame([list_roat_distance], columns=['trip', 'courier', 'day']), ignore_index=True)
    list_roat_distance = pd.DataFrame([list_roat_distance], columns=['trip', 'courier', 'day'])
    print(1)
    save_table(list_roat_distance, 'Sheet1')
    #print(list_roat_distance)
    return distance_array

def save_table(df, sheet):
    writer = pd.ExcelWriter('Отчет по курьерам.xlsx', engine='openpyxl')
    writer.book = load_workbook('Отчет по курьерам.xlsx')   
    writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
    reader = pd.read_excel(r'Отчет по курьерам.xlsx', sheet_name=sheet)
    df.to_excel(writer, sheet_name=sheet, index=False, header=False, startrow=len(reader)+1)
    writer.close()  
    

for i in adress_control['день + месяц'].unique():
    slice_first = adress_control[adress_control['день + месяц']==i]
    for j in slice_first['курьер'].unique():
        slice_second = slice_first[slice_first['курьер']==j].sort_values(by=['фактическое время доставки'])
        slice_second['адрес a'].iloc[0] = slice_second['торговое предприятие'].iloc[0]
        slice_second['адрес a lba'].iloc[0] = slice_second['торговое предприятие'].iloc[0]
        
        for z in range(len(slice_second)-1):
            if slice_second['фактическое время доставки'].iloc[z] < slice_second['время отправки доставки'].iloc[z+1]:
                slice_second['адрес a'].iloc[z+1] = slice_second['торговое предприятие'].iloc[z+1]
                slice_second['адрес a lba'].iloc[z+1] = slice_second['торговое предприятие'].iloc[z+1]
        
        for z in range(len(slice_second)):
            if slice_second['адрес a'].iloc[z] == 0:
                slice_second['адрес a'].iloc[z] = slice_second['coords b'].iloc[z-1]
                slice_second['адрес a lba'].iloc[z] = slice_second['адрес b'].iloc[z-1]
        
        #slice_second['адрес В'].iloc[len(slice_second)-1] = slice_second['торговое предприятие'].iloc[len(slice_second)-1]
        #slice_second['coords B'].iloc[len(slice_second)-1] = slice_second['торговое предприятие'].iloc[len(slice_second)-1]
                           
        adress_control_concat_table = pd.concat([adress_control_concat_table, slice_second], ignore_index=True)         

        slice_second['адрес a'] = slice_second['адрес a'].replace({'Алроса' : '52.951462 36.025452', 'Горького' : '52.978591 36.075605', 'Раздольная' : '53.014416 36.133214'})
        slice_second['coords b'] = slice_second['coords b'].replace({'Алроса' : '52.951462 36.025452', 'Горького' : '52.978591 36.075605', 'Раздольная' : '53.014416 36.133214'})
        slice_list = [list(a) for a in zip(slice_second['адрес a'].tolist(), slice_second['coords b'].tolist())]
        slice_list_A_B = [list(a) for a in zip(slice_second['адрес a lba'].tolist(), slice_second['адрес b'].tolist())] 
        #display(slice_list)
        temp_value = create_mass(slice_list, slice_list_A_B, j, i)
        
        save_table(temp_value, 'Sheet2')
        #display(otchet_about_couriers)
        #otchet_about_couriers = pd.concat([otchet_about_couriers], temp_value, ignore_index=True)
                
#display(otchet_about_couriers)
#display(finish_courier_trip


,index,торговое предприятие,время открытия,время печати накладной,время отправки доставки,планируемое время доставки,фактическое время доставки,время закрытия,курьер,время в пути(мин),...,ср.время в пути(мин),сумм.время в пути(мин),ср.опоздание доставки(мин),"сумма со скидкой, р.",день + месяц,адрес a,адрес b,адрес b sf,coords b,адрес a lba
0,0,Алроса,2022-06-14 10:23:11,2022-06-14 13:19:15.342,2022-06-14 13:21:58.147,2022-06-14 14:00:40.039,2022-06-14 13:48:47,2022-06-14 13:19:16.153,Евгений Егоров,26,...,26,26,0,1299,14.6.2022,0,г. Орел ул. 2-я Посадская д. 4 кв./оф. 41 под....,г. Орел ул. 2-я Посадская д. 4,52.957814 36.062391,0
1,1,Алроса,2022-06-14 10:40:50,2022-06-14 10:55:15.849,2022-06-14 11:02:29.700,2022-06-14 11:50:22.000,2022-06-14 11:13:38,2022-06-14 10:55:17.989,Конценебин Михаил,11,...,11,11,0,822,14.6.2022,0,г. Орел ул. Емлютина д. 12 кв./оф. 230 под. 6 ...,г. Орел ул. Емлютина д. 12,52.951516 36.013244,0
2,2,Алроса,2022-06-14 10:41:50,2022-06-14 12:10:11.339,2022-06-14 12:21:43.469,2022-06-14 13:00:01.000,2022-06-14 12:49:34,2022-06-14 12:10:12.338,Евгений Егоров,27,...,27,27,0,1079,14.6.2022,0,г. Орел ул. Машкарина д. 10 кв./оф. 68 под. 2 ...,г. Орел ул. Машкарина д. 10,52.925047 36.014142,0
3,3,Алроса,2022-06-14 10:44:52,2022-06-14 11:02:17.125,2022-06-14 11:02:28.025,2022-06-14 11:52:58.000,2022-06-14 11:25:56,2022-06-14 11:02:17.831,Конценебин Михаил,23,...,23,23,0,950,14.6.2022,0,г. Орел ул. Комсомольская д. 44 кв./оф. 27 под...,г. Орел ул. Комсомольская д. 44,52.958389 36.063954,0
4,4,Алроса,2022-06-14 11:33:12,2022-06-14 11:44:06.923,2022-06-14 11:44:29.946,2022-06-14 12:40:33.000,2022-06-14 12:06:28,2022-06-14 11:44:07.791,Евгений Егоров,21,...,21,21,0,925,14.6.2022,0,г. Орел ул. Орелстроевская д. 11А кв./оф. 145 ...,г. Орел ул. Орелстроевская д. 11А,52.93199 36.047632,0


C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-3-58a66e73df81>:73: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome('/Users/user/kvikfud/chromedriver_win32/chromedriver')


internal array


<ipython-input-3-58a66e73df81>:88: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  if ((float(browser.find_element_by_id("driving_status").text.split()[5])==0) and (flag < 2)):
<ipython-input-3-58a66e73df81>:92: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  distance_array.append(browser.find_element_by_id("driving_status").text.split()[5])
<ipython-input-3-58a66e73df81>:104: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  roat_distance = roat_distance + float(browser.find_element_by_id("driving_status").text.split()[5])


internal array
Нулевая дистанция
Нулевая дистанция
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
1
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
internal array
in

In [4]:
#Извлечение информации из файла с пробегом

import pandas as pd
dermo = pd.read_excel('д.xlsx')
print(dermo)
for i in range(len(dermo)-1):
    if dermo['Сотрудник'].iloc[i] == 'а':
        dermo['Сотрудник'].iloc[i] = dermo['Сотрудник'].iloc[i-1]

for i in range(len(dermo)-1):
    if dermo['Сотрудник'].iloc[i] == 'р':
        dermo['Сотрудник'].iloc[i] = dermo['Сотрудник'].iloc[i-1]

dermo = dermo[dermo['Волчков Сергей']!=0]

dermo.to_excel('д1.xlsx')

   Сотрудник  Волчков Сергей Сотрудник.1
0          1               0           1
1          а               0           а
2          р               0           р
3          2               0           2
4          а             108           а
..       ...             ...         ...
79         а               0           а
80         р               0           р
81        28               0          28
82         а               0           а
83         р               0           р

[84 rows x 3 columns]


C:\Users\user\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


FileCreateError: [Errno 13] Permission denied: 'д1.xlsx'

In [ ]:
import pandas as pd
dermo = pd.read_excel('д.xlsx')
print(dermo)
for i in range(len(dermo)-1):
    if dermo['Сотрудник'].iloc[i] == 'а':
        dermo['Сотрудник'].iloc[i] = dermo['Сотрудник'].iloc[i-1]

for i in range(len(dermo)-1):
    if dermo['Сотрудник'].iloc[i] == 'р':
        dermo['Сотрудник'].iloc[i] = dermo['Сотрудник'].iloc[i-1]

dermo = dermo[dermo['Волчков Сергей']!=0]

dermo.to_excel('д1.xlsx')

In [5]:
import pandas as pd
dermo1 = pd.read_excel('отчет для расчте за три месяца.xlsx')
dermo1['Day'] = dermo1['Время отправки доставки'].dt.date


In [47]:
grouped=dermo1.groupby(by = ['Курьер','Day'])['Курьер'].count()
g = dermo1.groupby(by = ['Курьер','Day'])['Курьер'].count()['Чикалин Сергей'].head(70)
for i,j in g.items():
    print(i,j)

2021-12-01 9
2021-12-02 16
2021-12-03 26
2021-12-06 8
2021-12-07 16
2021-12-08 5
2021-12-09 16
2021-12-10 15
2021-12-13 20
2021-12-14 10
2021-12-15 15
2021-12-16 14
2021-12-17 6
2021-12-20 8
2021-12-21 14
2021-12-22 6
2021-12-23 9
2021-12-24 21
2021-12-27 23
2021-12-28 24
2021-12-29 23
2021-12-30 26
2021-12-31 22
2022-01-03 26
2022-01-04 26
2022-01-05 25
2022-01-06 28
2022-01-07 20
2022-01-10 10
2022-01-11 13
2022-01-12 11
2022-01-13 24
2022-01-14 23
2022-01-17 10
2022-01-18 13
2022-01-19 20
2022-01-20 17
2022-01-21 22
2022-01-22 22
2022-01-24 13
2022-01-25 21
2022-01-27 15
2022-01-28 19
2022-01-29 28
2022-01-30 24
2022-01-31 14
2022-02-01 18
2022-02-03 16
2022-02-04 27
2022-02-05 24
2022-02-06 21
2022-02-07 15
2022-02-08 6
2022-02-09 17
2022-02-10 14
2022-02-11 18
2022-02-12 17
2022-02-13 17
2022-02-14 28
2022-02-15 12
2022-02-16 17
2022-02-17 18
2022-02-18 12
2022-02-19 20
2022-02-20 21
2022-02-21 9
2022-02-22 26
2022-02-23 36
2022-02-24 11
2022-02-25 12


In [26]:
df = pd.DataFrame([grouped])

In [29]:
df = df.unstack()

In [30]:
df

Курьер           Day               
Аничкин Виталий  2022-02-23  Курьер     7
                 2022-02-24  Курьер     8
                 2022-02-25  Курьер    13
                 2022-02-26  Курьер     8
                 2022-02-27  Курьер     9
                                       ..
курьер стажёр2   2022-02-12  Курьер    12
                 2022-02-13  Курьер    13
                 2022-02-14  Курьер    15
                 2022-02-21  Курьер     5
                 2022-02-22  Курьер    23
Length: 1465, dtype: int64